Author: Daniel Gallagher, 18401492

Data: NASA's Mars InSight Weather Data

# Need to include information about Sols, Mars Year

# Data from Red Skies
We haven't long been in this liminal state, pushing towards space exploration and discovery since only the last century. Desperate to escape the restrictions of life on Earth, we wish to go further than our *“pale blue dot”*. We've oft travelled towards things we felt bigger than ourselves barefoot, by sails, and on wheels. We now go a step further than that faithful day in July 1969 and look towards a red sky.

In 2011, NASA began the concept design for a mission to retrieve huge amounts of data from the rough surface of Mars. It would finally paint a picture for us of the deeply rich history of Mars' geological evolution. The data that we will be looking at was worth the cost of $830 million to NASA, and had many difficulties in its 7 year development.

This robotic lander was known as **InSight**.

![The InSight Lander on Mars](./images/lander_on_mars.png)

# Interior Exploration using Seismic Investigations, Geodesy and Heat Transport (or, uhm, InSight)

What has shaped this rocky planet of the interior Solar System over the last four billion years? This was the question on the minds of those designing each and every small detail on the Mars InSight robotic lander. It influenced every decision they made, and hence formed the attributes of the data we would receive from the red planet.

We will be examining data from a suite of sensors installed on the lander known as the [TWINS][1]. 

These sensors include:
* **Thermometers** for collecting atmospheric temperature information
* **An anemometer** for collecting wind speed and direction information
* **A highly sensitive barometer** for collecting atmospheric pressure information

Together, these tools will allow us to shape a fascinating picture of the Mars' climate and how it relates to our own.

[1]: https://en.wikipedia.org/wiki/Temperature_and_Winds_for_InSight

# Retrieving & Parsing InSight Data

We begin our data retrieval with requesting access to it. NASA allows us access through an **API key** which I signed up for on their website. Additionally, I have decided to use the [requests][1] library in order to retrieve this data from their server.

Let's begin by importing this library and specifying these important parameters for interaction with NASA.
[1]: https://requests.readthedocs.io/en/master/

In [217]:
import requests

#URL & Parameters 
url = 'https://api.nasa.gov/insight_weather/'
api_key = "fkkjfgveRAJ2BOVq7gaUAbBbM8omgKo0IRaDEGTj"
feedtype = "json"
ver = "1.0"

#Store these parameters in their own dictionary
params = dict(api_key=api_key, feedtype=feedtype, ver=ver)

In order to ensure that our request was successful, we will implement a function that checks the **status code** is in order. Thankfully, this is made easy for us.

In [218]:
def is_successful(response):
	return response.ok

Time to make a call to the server and request the InSight data!

In [219]:
#Send out our request and save the response
resp = requests.get(url=url, params=params)

#Ensure the response is what we're looking for
request_successful = is_successful(resp)
print ("Request Successful: %s" % (request_successful))

Request Successful: True


Perfect! We've successfully retrieved **something** from NASA.

However, that's about all we know at the moment. We need to explore the response further.

In [220]:
print "Response Type: %s" % (type(resp))

Response Type: <class 'requests.models.Response'>


Aha! Its a response! Wait... we already knew that...
Maybe we can explore this **Response** class and find out more about the data.

In [221]:
#The full URL that we retrieved the data from
url = resp.url

#The encoding of the information sent to us
enc = resp.encoding

#The time from first sending the request to receiving a response
time_elapsed = resp.elapsed

#The beginning of the data we've received 
start_of_data = resp.text[:200]

print ("URL: %s \n\nEncoding: %s \n\nTime Elapsed: %s \n\nBeginning Of Data:\n%s" %
      (url, enc, time_elapsed, start_of_data))

URL: https://api.nasa.gov/insight_weather/?ver=1.0&api_key=fkkjfgveRAJ2BOVq7gaUAbBbM8omgKo0IRaDEGTj&feedtype=json 

Encoding: utf-8 

Time Elapsed: 0:00:00.598070 

Beginning Of Data:
{
  "684": {
    "AT": {
      "av": -64.245, 
      "ct": 169658, 
      "mn": -95.453, 
      "mx": -17.861
    }, 
    "First_UTC": "2020-10-29T00:28:37Z", 
    "Last_UTC": "2020-10-30T01:08:12Z", 


That's more like it! We've learnt a good deal more about what we're working with.

Examine for a moment the beginning of the data. It gives us an important insight about the data; it is clearly in a **JSON format**. 
Our response class is excellent for handling this format, and we can thus easily create a dictionary object from this JSON object.

In [222]:
data = resp.json()
print ("Type: %s" % (type(data)))

Type: <type 'dict'>


We are entering the final step of our data retrieval & parsing process.

We would like to store this data in a JSON file for future use. 
Firstly, I'll create a function which will write response information to a file we pass in.
Our response class has an iterator object available through **iter_content** which will be useful here.

Next, I'll pass in where I would like to store this JSON file and run the function

In [223]:
def save_to_json_file(response, to_file):
	with open(to_file, "w") as f:
		for chunk in response.iter_content(chunk_size=128):
			f.write(chunk) 
		f.close()
        
json_file = "./json_storage/insight_data_storage.json"
save_to_json_file(resp, json_file) 

### Conclusion On Data Retrieval & Parsing
We began with a simple request out to NASA's InSight server and no knowledge about what we had received or if the request was successful.

We then ensured that NASA had responded to our request correctly. From this, we built up a knowledge of the type of data we had received and stored it in two separate ways based on this knowledge:
* We learned that it was given to us as JSON, and thus saved it to a python dictionary.
* We then saved the information to a JSON file for future use.

This concludes the process of data retrieval and parsing.


# Data Preprocessing & Integration

Data preprocessing is an incredibly important step in any data scientist's workflow. 
To have datasets which are poorly managed and cared for is to deal with a whole array of problems down the line. 

We will handle this step in three different stages:
* Data Cleaning & Reduction
* Data Transformation

When cultivated, these steps will lead to data from InSight that can be used confidently and efficiently.

We will use data saved from a previous call to NASA's server. This will be loaded into our **data** dictionary.

In [224]:
import json

previously_stored_json_path = "./json_storage/previously_stored_mars_data.json"

with open(previously_stored_json_path) as json_file:
    data = json.load(json_file)

Let's finally have a look at the attributes of our data.
We can do this by examining each **key** in our **data** dictionary.

In [225]:
data_attributes = []

for i,attr in enumerate(data):
    data_attributes.append(attr)
    print ("Attribute %d: %s" % (i+1,attr))

Attribute 1: 674
Attribute 2: 675
Attribute 3: 676
Attribute 4: 670
Attribute 5: 671
Attribute 6: 672
Attribute 7: 673
Attribute 8: validity_checks
Attribute 9: sol_keys


Ok, cool! We have some attributes. But what do they mean?

We can understand these attributes as:
* The previous seven sols (1 Mars Day = 1 Sol) 
* Ensuring we can be confident in the data through validity checks
* A list of each of the seven sol keys

However, none of these attributes tell us about our sensors! 
We want to see some barometer action.

Let's look inside a single Sol and see what attributes it presents to us.

In [226]:
#First, we'll save all Sols into a list
sols = data["sol_keys"]

#Choose the first sol as our example
example_sol = sols[0]
example_sol_data = data[example_sol]

print("Example Sol: %s\n" % (example_sol))

sol_attributes = []
for i,attr in enumerate(example_sol_data):
    sol_attributes.append(attr)
    print ("Attribute %d: %s" % (i+1,attr))

Example Sol: 670

Attribute 1: PRE
Attribute 2: WD
Attribute 3: Season
Attribute 4: Last_UTC
Attribute 5: AT
Attribute 6: First_UTC
Attribute 7: HWS


Here we see the aforementioned **TWINS sensor suite**:
* **PRE** - A barometer which feeds us our **atmospheric pressure** data.
* **WD & HWS** - An Anemometer which tells us about **wind direction and speed**, respectively.
* **AT** - Multiple thermometers which tell us about the **atmospheric temperature**.
* **Season** - Tells us what season it is on Mars.
* **First/Last UTC** - The time of the first and last datum transmitted on any given sol.

We now have a good idea of the important attributes within our dataset. This will guide us in the data cleansing process.

## Data Cleaning
Data cleaning is the process of removing any unwanted data from our dataset.
This includes errors and irrelevant information.
The NASA data received is very well kept, however we must ensure that there are no faults.

We will clean / scrub our data in these steps:
* Remove Unreliable/Missing Data
* Remove Irrelevant Information
* Quality Assurance

### Removing Unreliable / Missing Data Points
They say that the devil is in the sensors, or is it the details..? Either way, **we can be confident in our data if and only if we can be confident in our sensors**. That is, the barometer, anemometer, and thermometers.

NASA has made this very simple for us. Included in the data we received is an attribute called **validity_checks**. This attributes ensures our data is accurate by telling us how many data points were received by each sensor on each Sol (day on Mars). 

Let's examine the attributes of these validity checks for our sensors in our example sol.

In [227]:
#Store our validity data
validity_data = data["validity_checks"]

#Example Sol's Validity Checks 
example_validity = validity_data[example_sol]

#We only require validation for the sensors onboard.
sensors = ["PRE","AT","HWS","WD"]

#Iterate Through The Validity Checks Of Each Sensor
for i,sensor in enumerate(sensors):
    print ("Inspecting Validity Attributes Of %s" % (sensor))
    
    for attr in example_validity[sensor]:
        print ("Attribute: %s" % (attr))
    print('\n')

Inspecting Validity Attributes Of PRE
Attribute: valid
Attribute: sol_hours_with_data


Inspecting Validity Attributes Of AT
Attribute: valid
Attribute: sol_hours_with_data


Inspecting Validity Attributes Of HWS
Attribute: valid
Attribute: sol_hours_with_data


Inspecting Validity Attributes Of WD
Attribute: valid
Attribute: sol_hours_with_data




For each of our sensors, we can see NASA has included a **valid** boolean attribute in the **validity checks**. 
This attribute will equal **"True" if there are at least 18 hours on a given martian day with data sent from a sensor**. If there are less, it will equal "False".

To summarise, this validity check ensures:
* Data is present and not missing
* There is enough data to be considered reliable

This is an excellent example of **feature construction**. The InSight lander sent the data and NASA constructed a validity variable based on the consistency/reliability of these data. They then construct each of the variables we're working with in each sol around these data points, removing the possibility of missing data.

We can now validate each sensor by ensuring these **valid** attributes are true on each sol.

In [228]:
for sol in sols:
    validity = validity_data[sol]
    
    print("\nNext Sol: %s" % (sol))
    
    for sensor in sensors:
        is_valid = validity[sensor]["valid"]
        
        print("Validity of %s For Sol %s: %s" % (sensor,sol, is_valid))


Next Sol: 670
Validity of PRE For Sol 670: True
Validity of AT For Sol 670: True
Validity of HWS For Sol 670: True
Validity of WD For Sol 670: True

Next Sol: 671
Validity of PRE For Sol 671: True
Validity of AT For Sol 671: True
Validity of HWS For Sol 671: True
Validity of WD For Sol 671: True

Next Sol: 672
Validity of PRE For Sol 672: True
Validity of AT For Sol 672: True
Validity of HWS For Sol 672: True
Validity of WD For Sol 672: True

Next Sol: 673
Validity of PRE For Sol 673: True
Validity of AT For Sol 673: True
Validity of HWS For Sol 673: True
Validity of WD For Sol 673: True

Next Sol: 674
Validity of PRE For Sol 674: True
Validity of AT For Sol 674: True
Validity of HWS For Sol 674: True
Validity of WD For Sol 674: True

Next Sol: 675
Validity of PRE For Sol 675: True
Validity of AT For Sol 675: True
Validity of HWS For Sol 675: True
Validity of WD For Sol 675: True

Next Sol: 676
Validity of PRE For Sol 676: True
Validity of AT For Sol 676: True
Validity of HWS For Sol 

Fantastic, all our sensors are working and sending enough information to be considered reliable. They didn't spend hundreds of millions for nothing! 

## Remove Irrelevant Information

We can begin removing data that we don't want from our dataset. 

To begin, let's examine again the various attributes that we've been working with:

In [229]:
print("Data Attributes")
for attr in data_attributes:
    print(attr)
print('\n')
    
print("Individual Sol Attributes")
for attr in sol_attributes:
    print (attr)

Data Attributes
674
675
676
670
671
672
673
validity_checks
sol_keys


Individual Sol Attributes
PRE
WD
Season
Last_UTC
AT
First_UTC
HWS


No doubt there's a lot of interesting information here, but there's a difference between interesting and useful.
What can we prune away?

#### Pruning Data Attributes
Looking towards our data attributes, we definitely want to keep each individual sol. However, the validity checks are no longer useful as we have validated our data. These checks are purely for developers and are of no interest in extracting insights from our data further down the pipeline.

#### Pruning Sol Attributes
We would like to keep most of the attributes in each sol, as this is where we can really extract understanding about the nature of Mars' climate. However, we can remove the attributes which tell us the time of the first datum and last datum. This is perhaps interesting data, but not useful.

There is also the most_common attribute in each wind direction dataset. We want the all of the data wind direction data rather than just the most common direction during the day, therefore we will remove this attribute.


#### Final Prune
We have decided to remove the following unnecessary attributes from our dataset:
* Data - Validity Checks
* Sol  - First UTC
* Sol  - Last UTC
* Wind Dir - Most Common

In [230]:
#Remove Validity Checks & Sol Keys
data.pop("validity_checks", None)

#Remove Time Information
for sol in sols:
    data[sol].pop("First_UTC", None)
    data[sol].pop("Last_UTC", None)
    
    data[sol]["WD"].pop("most_common")

## Quality Assurance

Quality assurance involves making sure that our cleansing of the data has been successful. 
We will do this by ensuring that we have succesfully structured our dataset as we wanted in the previous example.

We will begin by ensuring that we removed the "validity_checks" attribute from the dataset.

In [231]:
#Ensure an attribute is no longer in our dataset
def is_removed(dataset, attr):
    return attr not in dataset

is_validity_removed = is_removed(data, "validity_checks")

print("Validity Checks Removed: %s" % (is_validity_removed))

Validity Checks Removed: True


We have successfully removed these unnecessary attributes.

Next, we want to ensure that we are only left with the sensors and the season attribute in each sol. Let's examine each sol to ensure this:

In [232]:
for sol in sols:
    print ("\nSol %s Attributes: %s" % (sol, data[sol].keys()))
    print ("Most Common Removed: %s\n" % ("most_common" not in data[sol]["WD"].keys()))


Sol 670 Attributes: [u'PRE', u'WD', u'Season', u'AT', u'HWS']
Most Common Removed: True


Sol 671 Attributes: [u'PRE', u'WD', u'Season', u'AT', u'HWS']
Most Common Removed: True


Sol 672 Attributes: [u'PRE', u'WD', u'Season', u'AT', u'HWS']
Most Common Removed: True


Sol 673 Attributes: [u'PRE', u'WD', u'Season', u'AT', u'HWS']
Most Common Removed: True


Sol 674 Attributes: [u'PRE', u'WD', u'Season', u'AT', u'HWS']
Most Common Removed: True


Sol 675 Attributes: [u'PRE', u'WD', u'Season', u'AT', u'HWS']
Most Common Removed: True


Sol 676 Attributes: [u'PRE', u'WD', u'Season', u'AT', u'HWS']
Most Common Removed: True



We have also successfully removed unnecessary attributes here.

This concludes our quality assurance for the dataset, and thus we have completed the data cleansing process. We can now be confident in the reliability and consistency of our data.

However, can we improve the structure of our data? Are we working with good database structure principles?

## Data Transformation 

Data transformation handles the way data are stored, maintained, and restrieved. We will be implementing a relational database management systems (RDBMS). Following certain rules, this will ensure good data management practices can be upheld.

In order to begin the restructuring of our data, we must implement a consistent procedure for accessing our data.
Accessing each table through a dictionary worked previously, but is not optimal.

We will begin by setting up an enum class that we can reference to find an ID for each sensor, rather than writing the string each time. This follows a good programming principle where nothing is "working only by coincedence".

In [233]:
#An enum to reference which table we're referring to
class Table(enum.Enum):
    Sols = 0
    Temp = 1
    W_Speed = 2
    W_Dir = 3
    Pressure = 4
    
table_ref = {
    Table.Sols : "Sols",
    Table.Temp : "AT",
    Table.W_Speed : "HWS",
    Table.W_Dir : "WD",
    Table.Pressure : "PRE"
}

def get_tbl_id(table):
    return table_ref[table]

It's time to begin setting up a new database.
We're going to use a mixture SQLite & Pandas in order to create a new database located in the "db" folder in our project.

We first want to create some functions that will allow us to very easily work with SQLite.

In [234]:
import pandas as pd
import sqlite3 as sql

def connect_to_db(name):
    return sql.connect("db/" + table_name)

#Save a table to the relational database
def save_data(name, rows, columns):
    db_conn = connect_to_db(name)
    df = pd.DataFrame(data=rows, columns=columns)
    df.to_sql(name,db_conn,index=False,if_exists='replace')
    db_conn.close()

#Retrieve an entire table from the database
def get_table(tbl):
    db_conn = connect_to_db(tbl)
    sql_cmd = "SELECT * FROM %s" % (tbl)
    df = pd.read_sql(sql_cmd,db_conn)
    db_conn.close()
    return df

#Execute a custom SQL query on a table
def query_table(tbl,query):
    db_conn = connect_to_db(tbl)
    cur = db_conn.cursor()
    exe = cur.execute(query)
    rows = [row for row in exe]
    db_conn.close()
    return rows

It's important that we create proper relations between different tables and follow good relational database practices.

Below, I have laid out the structure that I will be setting up for my data in this project.
There is a table for each sensor and any additional tables required for making relations between tables.
We can see that there is one-to-many relationship for our sols. Note each of the unique keys for our table. 

Originally I wondered if it would be suitable to set up a table for ordinals on the compass, to give them a proper ID. However, the number of the ordinal is already the perfect ID and there is no other information required. Therefore, a table was not created to store IDs for ordinal numbers.

![Relational Database Model For Mars Data](./images/db_structure.png)



### Table Time! Setting up the Sols table

Let's have another look at the current attributes of our data.

In [235]:
print ([str(x) for x in data.keys()])

['674', '675', '676', '670', '671', '672', '673', 'sol_keys']


We can see that our data is currently grouped by sol.

This is not an optimal implementation. We want to create a unique sol ID for each sol and store it in its own table along with the particular day of the year as we see above. This table will then be used for a one-to-many relationship with other tables grouped by subject.

Let us begin with our new table for sols.

In [236]:
"""
Sols Table
Name: SOLS
2 Attributes: Unique Sol ID | Number of Days into the Year
"""
table_name = get_tbl_id(Table.Sols)

#The data we're already given about sols
days_into_year = data["sol_keys"]

#Unique Sol ID: Starting from 0
unique_ids = [i for i in range(0,len(days_into_year))]

#Attach our two rows together
rows = zip(unique_ids, days_into_year)

#Create the columns
cols = ["sol_id", "days_into_year"]

#Save our data to our relational database
save_data(table_name, rows, cols)

get_table(table_name)

,sol_id,days_into_year
0,0,670
1,1,671
2,2,672
3,3,673
4,4,674
5,5,675
6,6,676


This above table will store the IDs we need for creating primary keys in the tables for each sensor.
We can create a few functions which will make the process of table creation much easier for the sensors.

The function **structure_sensor_data** will accept a sensor ID and create a table of everything that sensor picked up over the last seven days on Mars. Our other function will convert from *sol id* to *day of the year*. 

For example, ID 0 => Day 612

In [237]:
#Last seven days on Mars
recent_sols = get_table(get_tbl_id(Table.Sols))["sol_id"]

#Convert Sol ID => Day Of The Year
def id_to_sol(sol_id):
    query = "SELECT days_into_year FROM Sols WHERE sol_id=%s" % (sol_id)
    tbl = "Sols"
    return str(query_table(tbl,query)[0][0])

def structure_sensor_data(sensor_id, recent_sols):
    sensor_rows = []
    sensor_cols = ["sol_id", "av", "mn", "mx", "ct"]
    
    for sol_id in recent_sols:
        day = id_to_sol(sol_id)
        
		#Retrieve Sensor Data For Day x
        sensor_row = data[day][sensor_id]
        
        #Add the Sol ID to the row
        sensor_row["sol_id"] = sol_id
        
        sensor_rows.append(sensor_row)

    return sensor_rows, sensor_cols

In [238]:
#Atmospheric Temperature
table_id = get_tbl_id(Table.Temp)
atm_temp_rows, atm_temp_cols = structure_sensor_data(table_id,recent_sols)
save_data(table_id, atm_temp_rows, atm_temp_cols)

#Wind Speed
table_id = get_tbl_id(Table.W_Speed)
wspeed_rows, wspeed_cols = structure_sensor_data(table_id, recent_sols)
save_data(table_id, wspeed_rows, wspeed_cols)

#Atmospheric Pressure
table_id = get_tbl_id(Table.Pressure)
pre_rows, pre_cols = structure_sensor_data(table_id,recent_sols)
save_data(table_id, pre_rows, pre_cols)

#View Our Tables
print(get_table(get_tbl_id(Table.Temp)))
print(get_table(get_tbl_id(Table.W_Speed)))
print(get_table(get_tbl_id(Table.Pressure)))

   sol_id      av      mn      mx      ct
0       0 -60.662 -95.821 -15.826  309399
1       1 -62.760 -95.959  -8.545  351830
2       2 -67.455 -96.828  -6.506  223227
3       3 -62.386 -96.654  -9.915  177556
4       4 -57.602 -96.011  -7.439  259074
5       5 -63.280 -96.872 -15.908  248874
6       6 -66.826 -96.912 -16.499  181986
   sol_id     av     mn      mx      ct
0       0  7.915  0.527  23.077  153433
1       1  5.678  0.246  19.108  171610
2       2  4.741  0.214  15.995  106953
3       3  5.636  0.191  18.862   86796
4       4  5.422  0.235  18.469  126068
5       5  7.295  1.051  22.455  123240
6       6  8.853  1.110  26.905   90466
   sol_id       av        mn        mx      ct
0       0  750.909  723.9181  771.5430  152448
1       1  748.289  722.8493  764.1114  150704
2       2  749.078  723.3106  766.2122  156607
3       3  747.691  721.0958  766.4046   88773
4       4  746.625  722.3939  764.9635  129594
5       5  751.467  722.7706  768.7766  123307
6       6  750.

Lastly, we must structure our wind direction table.

This table is a bit different, as we're working with directions of a compass which we refer to as the ordinals. These ordinals will be apart of our super key {sol_id,ordinal} for this table.

In [239]:
#Wind Direction: Ordinals
#Relation Key: {sol_id, ordinal_id}

table_id = get_tbl_id(Table.W_Dir)

wd_rows = []

"""
The columns for this table will be
very different to the other tables.
"""
wd_cols = ["sol_id", "ordinal_id", "compass_point", "compass_degrees", 
"compass_right", "compass_up", "ct"]

for sol in recent_sols:
    day = id_to_sol(sol)

    #Get all of our compass data for that day 
    ds = data[day]["WD"]
    ordinals = ds.keys()

    #For each direction on a wind rose
    for ordinal in ordinals:
        #Row for Day x, Ordinal y
        next_row = ds[ordinal]
        
        #Super Key: { sol_id, ordinal_id }
        next_row["sol_id"] = sol
        next_row["ordinal_id"] = ordinal

        wd_rows.append(next_row)

save_data(table_id, wd_rows, wd_cols)

#Print the first 10 entries in our table
print(get_table(get_tbl_id(Table.W_Dir))[:10])

   sol_id ordinal_id compass_point  compass_degrees  compass_right  \
0       0         11           WSW            247.5      -0.923880   
1       0         10            SW            225.0      -0.707107   
2       0         13           WNW            292.5      -0.923880   
3       0         12             W            270.0      -1.000000   
4       0         15           NNW            337.5      -0.382683   
5       0         14            NW            315.0      -0.707107   
6       0          1           NNE             22.5       0.382683   
7       0          0             N              0.0       0.000000   
8       0          5           ESE            112.5       0.923880   
9       0          9           SSW            202.5      -0.382683   

   compass_up     ct  
0   -0.382683   6120  
1   -0.707107   9572  
2    0.382683  57611  
3    0.000000  40009  
4    0.923880   5026  
5    0.707107  33779  
6    0.923880     34  
7    1.000000   1268  
8   -0.382683      2  

### Conclusion Of Data Pre-Processing & Integration

This concludes the numerous stages of pre-processing and integration with the NASA InSight lander data.

One of the first things that I learned studying data science was the importance of this step. 
You could say that skimping on this stage of data analysis is like entering a fight without having gone to the gym. You might be able to swing around a bit, but the potential for great work is limited.

Maybe NetSoc & UCD Boxing should collaborate. Anyway...

Time to gain some actual insights from our data. We've seen some numbers floating (excuse the pun) about, but haven't told the story the data provides for us. 

This next stage of data analysis is worth 40 percent of this assignment, and thus must be thorough. I'm going to ask five questions, and give you five answers. In between each, I'm going to see what tale the data can weave, and hopefully they give us insights into the story of Mars. At the core of each question, I would like to find a tangible insight. Let's go!